# 模型训练

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_train.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_train.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/beginner/model_train.ipynb)

通过上面章节的学习，我们已经学会如何创建模型和构建数据集，现在开始学习如何设置超参和优化模型参数。

## 超参（Hyper-parametric）

超参是可以调整的参数，可以控制模型训练优化的过程，不同的超参数值可能会影响模型训练和收敛速度。目前深度学习模型多采用批量随机梯度下降算法进行优化，随机梯度下降算法的原理如下：

$$w_{t+1}=w_{t}-\eta \frac{1}{n} \sum_{x \in \mathcal{B}} \nabla l\left(x, w_{t}\right)$$

式中，$n$是批量大小(batch size)，$η$是学习率(learning rate)；另外，$w_{t}$为训练轮次t中权重参数，$\nabla l$为损失函数的导数。可知道除了梯度本身，这两个因子直接决定了模型的权重更新，从优化本身来看它们是影响模型性能收敛最重要的参数。一般会定义以下超参用于训练：

训练轮次（epoch）：训练时遍历数据集的次数。

批次大小（batch size）：数据集进行分批读取训练，设定每个批次数据的大小。batch size过小，花费时间多，同时梯度震荡严重，不利于收敛；batch size过大，不同batch的梯度方向没有任何变化，容易陷入局部极小值，因此需要选择合适的batch size，可以有效提高模型精度、全局收敛。

学习率（learning rate）：如果学习率偏小，会导致收敛的速度变慢，如果学习率偏大则可能会导致训练不收敛等不可预测的结果。梯度下降法是一个广泛被用来最小化模型误差的参数优化算法。梯度下降法通过多次迭代，并在每一步中最小化损失函数来估计模型的参数。学习率就是在迭代过程中，会控制模型的学习进度。

![learning_rate](images/learning_rate.png)

In [19]:
epochs = 1
batch_size = 32
learning_rate = 1e-3

## 损失函数

**损失函数**用来评价模型的**预测值**和**真实值**之间的误差，在这里，使用绝对误差损失函数`L1Loss`。`mindspore.nn.loss`也提供了许多其他常用的损失函数，如`SoftmaxCrossEntropyWithLogits`、`MSELoss`、`SmoothL1Loss`等。

我们给定输出值和目标值，通过损失函数计算预测值和目标值之间的误差（损失值），使用方法如下所示：

In [20]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor

loss = nn.L1Loss()
output_data = Tensor(np.array([[1, 2, 3], [2, 3, 4]]).astype(np.float32))
target_data = Tensor(np.array([[0, 2, 5], [3, 1, 1]]).astype(np.float32))

print(loss(output_data, target_data))

1.5


## 优化器函数

优化器函数用于计算和更新梯度，模型优化算法的选择直接关系到最终模型的性能。有时候最终模型效果不好，未必是特征或者模型设计的问题，很有可能是优化算法的问题。

MindSpore所有优化逻辑都封装在`Optimizer`对象中，在这里，我们使用`Momentum`优化器。`mindspore.nn`也提供了许多其他常用的优化器函数，如`Adam`、`SGD`、`RMSProp`等。

我们需要构建一个`Optimizer`对象，这个对象能够基于计算得到的梯度对参数进行更新。为了构建一个`Optimizer`，需要给它一个包含可优化的参数（必须是Variable对象）的迭代器，如网络中所有可以训练的`parameter`，将`params`设置为`net.trainable_params()`即可。

然后，设置`Optimizer`的参数选项，比如学习率、权重衰减等。代码样例如下：

In [21]:
from mindspore import nn
from mindvision.classification.models import lenet

net = lenet(num_classes=10, pretrained=False)
optim = nn.Momentum(net.trainable_params(), 0.1, 0.9)

## 模型训练

模型训练一般分为四个步骤：

1. 构建数据集。
2. 定义神经网络。
3. 定义超参、损失函数及优化器。
4. 输入训练轮次和数据集进行训练。

模型训练示例代码如下：

In [ ]:
import mindspore.nn as nn
from mindspore import context
from mindspore.train import Model

from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet
from mindvision.engine.callback import LossMonitor

context.set_context(mode=context.GRAPH_MODE)

# 1. 构建数据集
download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
dataset_train = download_train.run()

# 2. 定义神经网络
network = lenet(num_classes=10, pretrained=False)
# 定义损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
# 3.1 定义优化器函数
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)
# 3.2 初始化模型参数
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={'acc'})

# 4. 对神经网络执行训练
model.train(epochs, dataset_train, callbacks=[LossMonitor(125)])

## 保存模型

在训练完网络完成后，下面将会讲述下如何保存和加载模型，保存模型的接口有主要2种方式：

一种是简单的对网络模型进行保存，可以在训练前后进行保存，优点是接口简单易用，但是只保留执行命令时候的网络模型状态；

另外一种是在网络模型训练中进行保存，MindSpore在网络模型训练的过程中，自动保存训练时候设定好的epoch数和step数的参数，也就是把模型训练过程中产生的中间权重参数也保存下来，方便进行网络微调和停止训练。

### 直接保存模型

使用MindSpore提供的save_checkpoint保存模型，传入网络和保存路径：

In [ ]:
import mindspore as ms

# 定义的网络模型为net，一般在训练前或者训练后使用
ms.save_checkpoint(network, "./MyNet.ckpt")

其中，`net`为训练网络，定义方法可参考[建立神经网络](https://www.mindspore.cn/tutorials/zh-CN/master/model.html)。

### 训练过程中保存模型

在模型训练的过程中，使用`model.train`里面的`callbacks`参数传入保存模型的对象 `ModelCheckpoint`，可以保存模型参数，生成CheckPoint(简称ckpt)文件。
其中，`epoch_num`为训练轮次，定义方法可参考[训练模型](https://www.mindspore.cn/tutorials/zh-CN/master/optimization.html)。`dataset`为加载的数据集，定义方法可参考[数据加载及处理](https://www.mindspore.cn/tutorials/zh-CN/master/dataset.html)。

用户可以根据具体需求对CheckPoint策略进行配置。具体用法如下：

In [12]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

# 设置epoch_num数量
epoch_num = 1
# 设置模型保存参数
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
# 应用模型保存参数
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)
model.train(epoch_num, dataset_train, callbacks=ckpoint, dataset_sink_mode=False)

上述代码中，首先需要初始化一个`CheckpointConfig`类对象，用来设置保存策略。

- `save_checkpoint_steps`表示每隔多少个step保存一次。
- `keep_checkpoint_max`表示最多保留CheckPoint文件的数量。
- `prefix`表示生成CheckPoint文件的前缀名。
- `directory`表示存放文件的目录。

创建一个`ModelCheckpoint`对象把它传递给`model.train`方法，就可以在训练过程中使用CheckPoint功能了。

生成的CheckPoint文件如下：

```text
lenet-graph.meta # 编译后的计算图
lenet-1_1875.ckpt  # CheckPoint文件后缀名为'.ckpt'
lenet-1_1875.ckpt  # 文件的命名方式表示保存参数所在的epoch和step数
lenet-1_1875.ckpt  # 表示保存的是第3个epoch的第32个step的模型参数
...
```

如果用户使用相同的前缀名，运行多次训练脚本，可能会生成同名CheckPoint文件。MindSpore为方便用户区分每次生成的文件，会在用户定义的前缀后添加"_"和数字加以区分。如果想要删除`.ckpt`文件时，请同步删除`.meta` 文件。

例：`lenet_3-2_1875.ckpt` 表示运行第3次脚本生成的第2个epoch的第32个step的CheckPoint文件。

## 加载模型

要加载模型权重，需要先创建相同模型的实例，然后使用`load_checkpoint`和`load_param_into_net`方法加载参数。

示例代码如下：

In [ ]:
from mindspore import load_checkpoint, load_param_into_net
from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet

# 将模型参数存入parameter的字典中，这里加载的是上面训练过程中保存的模型参数
param_dict = load_checkpoint("./lenet/lenet-1_1875.ckpt")
# 将参数加载到网络中
load_param_into_net(network, param_dict)
model = Model(network, net_loss, metrics={"accuracy"})

- `load_checkpoint`方法会把参数文件中的网络参数加载到字典`param_dict`中。
- `load_param_into_net`方法会把字典`param_dict`中的参数加载到网络或者优化器中，加载后，网络中的参数就是CheckPoint保存的。

### 模型验证

针对仅推理场景，把参数直接加载到网络中，以便后续的推理验证。示例代码如下：

In [14]:
# 调用eval()进行推理
acc = model.eval(dataset_eval, dataset_sink_mode=False)
print("{}".format(acc))

{'accuracy': 0.9818709935897436}


### 用于迁移学习

针对任务中断再训练及微调（Fine-tuning）场景，可以加载网络参数和优化器参数到模型中。示例代码如下：

In [15]:
# 定义训练数据集
download_train = Mnist(path="./mnist", split="train", batch_size=32, repeat_num=1, shuffle=True, resize=32, download=True)
dataset_train = download_train.run()

# 调用train()进行训练
model.train(epoch_num, dataset_train, dataset_sink_mode=False)